##   <font color='Lime'>Import libraries</font> 

In [ ]:
import os
import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import backend as K
!pip install segmentation-models
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# Set some parameters
im_width = 128
im_height = 128
im_chan = 1
#path_train = r'C:\\Users\diego\OneDrive\Desktop\DS\seismic_segmentation\train'
#path_test = r'C:\\Users\diego\OneDrive\Desktop\DS\seismic_segmentation\test'

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

##   <font color='Lime'>Data Analysis</font> 


In [ ]:
import zipfile

path_train = r'../input/tgs-salt-identification-challenge/train.zip'
path_test = r'../input/tgs-salt-identification-challenge/test.zip'

unziped_train= '/kaggle/working/unziped_train'
unziped_test= '/kaggle/working/unziped_test'
with zipfile.ZipFile("../input/tgs-salt-identification-challenge/train"+".zip","r") as z:
    z.extractall(unziped_train)
with zipfile.ZipFile("../input/tgs-salt-identification-challenge/test"+".zip","r") as z:
    z.extractall(unziped_test)    


In [ ]:
'''
ids= ['1f1cc6b3a4','5b7c160d0d','6c40978ddf','7dfdf6eeb8','7e5a6e5013']
plt.figure(figsize=(20,10))
for j, img_name in enumerate(ids):
    q = j+1
    img = load_img('./masks' + img_name + '.png')
    img_mask = load_img('./images' + img_name + '.png')
    
    plt.subplot(1,2*(1+len(ids)),q*2-1)
    plt.imshow(img)
    plt.subplot(1,2*(1+len(ids)),q*2)
    plt.imshow(img_mask)
plt.show()'''

In [ ]:
import os
train_ids=[]
test_ids=[]
for i in os.listdir(os.path.join(unziped_train,"images")):
    train_ids.append(i)
for i in os.listdir(os.path.join(unziped_test,"images")):
    test_ids.append(i)    

In [ ]:
# Get and resize train images and masks
import numpy as np
X_train1 = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_train1 = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = unziped_train
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    X_train1[n] = x
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    Y_train1[n] = resize(mask, (128, 128, 1), mode='constant', preserve_range=True)

print('Done!')

In [ ]:
# Check if training data looks all right
ix = random.randint(0, len(train_ids))
plt.imshow(np.dstack((X_train1[ix],X_train1[ix],X_train1[ix])))
plt.show()
tmp = np.squeeze(Y_train1[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()

In [ ]:
def Intersection_over_Union(y_true, y_pred, threshold=0.5):
    y_pred = K.squeeze(tf.cast(y_pred > threshold,tf.int32), -1)
    y_true = K.cast(y_true[..., 0], K.floatx())
    y_pred = K.cast(y_pred, K.floatx())
    truth_areas = K.sum(y_true, axis=[1, 2])
    pred_areas = K.sum(y_pred, axis=[1, 2])
    intersection = K.sum(y_true * y_pred, axis=[1, 2])
    union = K.clip(truth_areas + pred_areas - intersection, 1e-9, 128 * 128)
    check = K.map_fn(lambda x: K.equal(x, 0), truth_areas + pred_areas, dtype=tf.bool) #images with no predicted and true salt bodies
    p = intersection / union #ratio for each image
    iou = K.switch(check, p + 1., p) # put 1 for images where no salt bodies were found 

    prec = K.map_fn(lambda x: K.mean(K.greater(x, np.arange(0.5, 1.0, 0.05))), iou, dtype=tf.float32) 
    #does each image have a iou greater than threshold? if so, the image iou is used to calculate the mean
    prec_iou = K.mean(prec)
    return prec_iou

##  <font color='Lime'>Data Augmentation</font> 

In [ ]:
# Get and resize test images
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = unziped_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    X_test[n] = x

print('Done!')

In [ ]:
#step=n*(#images/batch_size)
#learning_rate = initial_learning_rate * decay_rate ^ (step / decay_steps)
0.000018 * 0.9**(23*(3600/20) / 1000)

In [ ]:
from albumentations import (HorizontalFlip, ShiftScaleRotate, RandomContrast, RandomBrightness, Compose,Rotate)
import albumentations as A
import cv2
import pickle
from sklearn.model_selection import train_test_split

#tf.random.set_seed(42)
#
#import keras
#from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
import segmentation_models as sm
sm.set_framework('tf.keras')


checkpoint=r"models/checkpoint"
if  os.path.exists(os.path.join(os.getcwd(),checkpoint))==False:
    if  os.path.exists(os.path.join(os.getcwd(),'models'))==False:
        os.mkdir('models/')
    os.mkdir(checkpoint)

from tensorflow.keras.optimizers import schedules
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.000029,
    decay_steps=6000,
    decay_rate=0.89
    )


opt = Adam(
    learning_rate=lr_schedule, # add exponential decay
    epsilon=1e-08)

sm.framework()
nets = 15
model = [0] *nets
N=1
backbone='resnet34'
best_models=[]
for j in range(nets):
    
    base_model = sm.Unet(backbone, encoder_weights='imagenet')# classes=1, activation='sigmoid')
    inp = Input(shape=(None, None, N))
    s = Lambda(lambda x: x / 255)(inp)
    l1 = tf.keras.layers.Conv2D(3, (1, 1))(s) # map N channels data to 3 channels
    out = base_model(l1)

    model[j] = Model(inp, out, name=base_model.name)
    model[j].compile(optimizer=opt, loss='binary_crossentropy', metrics=[Intersection_over_Union])# metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

    # summarize the model

    model[j].summary()

    #model_checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(os.getcwd(),checkpoint),
       # save_weights_only=True,
       # monitor='val_Intersection_over_Union',
        #mode='max',
       # save_best_only=True)

earlystopper = EarlyStopping(patience=7,monitor='val_Intersection_over_Union', mode='max'
                             , verbose=1,restore_best_weights=True)
histories=[]

#checkpointer = ModelCheckpoint('model-tgs-salt-1.h5', verbose=1, save_best_only=True)
for j in range(nets):
    X_train, X_val, y_train, y_val = train_test_split(X_train1, Y_train1, test_size = 0.1)
    transform = A.Compose([
        HorizontalFlip(p=0.5)])
    transformed = transform(image=np.squeeze(X_train), mask=np.squeeze(y_train))
    history = model[j].fit( np.expand_dims(transformed['image'],axis=3),np.expand_dims(transformed['mask'],axis=3),validation_data = (X_val,y_val)
                              , batch_size=20, epochs=26, callbacks=[earlystopper])
    histories.append(history)
    print(os.getcwd())

    #save the model
    
    name=str(j)+'CNN_Unet_onlyflip_exponentialdecay_cross_entropy_early_stop_IoU_ensemble'
    
    
    #with open("y_train", "wb") as f:
       # pickle.dump(y_train, f)
    
    #os.mkdir('models/')
    if  os.path.exists(os.path.join(os.getcwd(),'models/'))==False:
        os.mkdir('models/')
    #model[j].save('models/'+name+'.h5')
    
    #plt.gca().set_xlim(0,33)


    if  os.path.exists(os.path.join(os.getcwd(),'plots/'))==False:
        os.mkdir('plots/')

    
     
    
    pd.DataFrame(history.history).plot(figsize=(8, 5))

    plt.title('Model Loss and Accuracy',fontsize=14,y=1.03 )
    plt.grid( True )    

    plt.ylabel('Loss and Accuracy',labelpad=7,fontsize=10)    
    plt.xlabel('Epochs',labelpad=7,fontsize=10)   
    plt.gca().set_ylim(0,1)
    plt.savefig(os.path.join('plots',name+'.png'), dpi=500, bbox_inches='tight',facecolor=(1, 1, 1) )

    
    #plot loss and accuracy
    loss, accuracy = model[j].evaluate(X_val,y_val)
    
    
    #plt.gca().set_ylim(0,1)
    #plt.grid(True)
    if  os.path.exists(os.path.join(os.getcwd(),'models/details'))==False:
        os.mkdir('models/details')
    with open(os.path.join('models/details',name+'.txt'), 'w+') as f:
        f.write(f'{name} \n  \n ')
        f.write(f'accuracy : {round(accuracy*100,4)} \n loss : {round(loss,4)}  \n ')
        f.write(f' \n optimizer: \n {model[j].optimizer.get_config()}')
    #evaluate loss and accuracy
    threshold=0.65
    if  os.path.exists(os.path.join(os.getcwd(),'models/accurate'))==False:
        os.mkdir('models/accurate')
    if accuracy >threshold:
        model[j].save_weights('models/accurate'+name+'.h5')
        with open(name, "wb") as f:
            pickle.dump(model[j].predict(X_test), f)
        best_models.append(j)
    print('Accuracy: %f' % (accuracy*100),'Loss: %f' % (loss))    


#print accuracy 


#X_train,X_val,y_train,y_val = train_test_split(X_train,Y_train, test_size=0.1)


In [ ]:
#results = np.zeros( X )
#for j in range(nets):
    #results = results + model[j].predict(X_test)
#results = np.argmax(results,axis = 1)
#results = pd.Series(results,name="Label")
#results


In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
#FileLink('./models/accurate10CNN_Unet_onlyflip_exponentialdecay_cross_entropy_early_stop_IoU_ensemble.h5')
FileLink('0CNN_Unet_onlyflip_exponentialdecay_cross_entropy_early_stop_IoU_ensemble')

In [ ]:
NotebookApp.iopub_msg_rate_limit=8000.0

In [ ]:
# ENSEMBLE PREDICTIONS AND SUBMIT
results = []
for j in range(nets):
    results = results + model[j].predict(X_val)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
results

In [ ]:
X_val.shape

##  <font color='Lime'>Train the model</font> 


##  <font color='Lime'>Save the model and plot accuracy and loss</font> 

In [ ]:

model.load_weights(os.path.join('models/',name+'_weights.h5'))
model.evaluate(X_val,y_val)

In [ ]:
pd.DataFrame(history.history)

##  <font color='Lime'>Accuracy and Loss</font> 

In [ ]:
print(f'accuracy : {round(accuracy*100,3)} \n loss : {round(loss,3)}')

In [ ]:
# visualize activation functions
for i, layer in enumerate (model.layers):
    print (i, layer)
    try:
        print ("    ",layer.activation)
    except AttributeError:
        print('   no activation attribute')
#specific info about each layer
for i in range(len(model.layers)):
    print(f'{i}   {model.layers[i]}: \n{model.layers[i].get_config()} \n')
#info about optimizers
model.optimizer.get_config()  

##  <font color='Lime'>Load the model</font> 

In [ ]:
loss, accuracy = saved_model.evaluate(X_val,y_val)

In [ ]:
#here I plotted the images based on columns and z_max (defined by the user, but it should be a multiple of the # of columns)
#columns = 20
n_images=100
columns=10


images=X_val[:n_images]


#int(len(images)/columns)

fig,axes=plt.subplots(int(len(images)/columns),columns,figsize=(40,20))
fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.463, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
for i,ind in enumerate(images):
    #print(images_class)
    fig.suptitle(f'First {n_images} sections in the dataset',fontsize=26, x=0.2812,y=0.946)
    #print(j,image)
    axes[int(i/columns), i % columns].imshow(ind)
    #axes[int((i)*(len(ind)/columns)+(int(j/columns))),j % columns] #.set_title(f'{classes[i]}',pad=10)
#image_plotting(X_test[:1])
plt.savefig(os.path.join('plots/','100_images_in_the_dataset_white.png'), dpi=300,facecolor='w',bbox_inches='tight')

##  <font color='Lime'>Test Data</font> 

In [ ]:
for j in range(nets):
    pred=model[j].predict(X_test)
    np.save(os.path.join('models',str(j)),pred)

In [ ]:
from IPython.display import FileLink  
os.chdir(r'/kaggle/working')
#FileLink(r'models/'+str(0)+'.npy')
#FileLink(r'models/'+str(1)+'.npy')
#FileLink(r'models/'+str(2)+'.npy')
#FileLink(r'models/'+str(3)+'.npy')
#FileLink(r'models/'+str(4)+'.npy')
#FileLink(r'models/'+str(5)+'.npy')
#FileLink(r'models/'+str(6)+'.npy')

In [ ]:
results = []
for j in range(nets):
    pred=np.load(os.path.join('models',str(j)+'.npy'))
    results.append(pred)
#results = np.argmax(results,axis = 1)
#results = pd.Series(results,name="Label")
#results

In [ ]:
preds_test = model.predict(X_test, verbose=1)

preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
# Create list of upsampled test masks
preds_test_upsampled = []
for i in tnrange(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
preds_test_upsampled[0].shape

##  <font color='Lime'>Submission</font> 


In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission_unet_cross_entropy_lovasz.csv')